Вариант 7

4.1.7, 4.2, 4.3, 4.6.2

In [1]:
import numpy as np
from numpy.linalg import solve, norm, cond
import copy, sys

## 4.1.7

#### 1.

In [2]:
A = np.array([[4.95, 1.12, 2.9, 0.66],
              [8.91, 19.9, -4.0, 6.93],
              [-2.97, 2.2, -5.8, 0],
              [5.94, 1.3, 10.5, 17.82]],
             dtype=np.float32).reshape([4, 4])
b = np.array([-3.41, 50.33, 19.49, -45.88], dtype=np.float32)
x_prec = solve(A, b)
x_prec

array([ 0.77777755,  2.0000005 , -2.9999995 , -1.2121215 ], dtype=float32)

#### 2.

In [3]:
def modify(A, b):
    n = len(A)
    B = np.zeros(shape=(n,n))
    c = np.zeros(shape=(n,))
    for i in range(n):
        c[i] = b[i]/A[i, i]
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j]/A[i,i]
    return B, c
def get_b2(B):
    B2 = copy.deepcopy(B)
    m = len(B2)
    for i in range(m):
        for j in range(i+1):
            B2[i, j] = 0
    return B2

In [4]:
B, c = modify(A, b)
norm(B, ord=np.inf)

0.9969849437475204

#### 3.

In [5]:
def zeidel(A, b, x0, n_max, eps, sparse=False):
    B, c = modify(A, b)
    m = x0.shape[0]
    x = np.zeros(m)
    x_prev = copy.deepcopy(x0)  
    n_iter = 0
    n_iter_prev = -1
    B2 = get_b2(B)
    eps = (1-norm(B, ord=np.inf))/norm(B2, ord=np.inf)*eps
    while(n_iter != n_max):
        n_iter_prev = n_iter
        n_iter+=1
        assert (n_iter<sys.maxsize-1), "Too many iterations without convergence"
        for i in range(m):
            x[i] = sum([B[i, j]*x[j] for j in range(0, i)]) + \
            sum([B[i, j]*x_prev[j] for j in range(i+1, m)]) + \
            c[i]
        delta = norm(x - x_prev, ord=np.inf)
        if delta < eps:
            return x, n_iter, delta
        x_prev = copy.deepcopy(x)    
    return x, n_iter, delta

#### 4.

In [6]:
x0_1 = np.array([1, 1, 1, 1]).reshape([-1,])
x_zd_1 = zeidel(A, b, x0_1, n_max=10, eps=10**-11)[0]
x_zd_1

array([ 0.74463897,  2.02722881, -2.97270228, -1.21914588])

In [7]:
norm(x_zd_1 - x_prec, ord=np.inf)

0.03313858610631426

#### 5.

In [8]:
x0_2 = np.array([0.7, 2, -2.9, -1.2]).reshape([-1,])
x_zd_2 = zeidel(A, b, x0_2, n_max=10, eps=10**-11)[0]
x_zd_2

array([ 0.77676934,  2.00082892, -2.999169  , -1.21233522])

In [9]:
norm(x_zd_2 - x_prec, ord=np.inf)

0.00100820767923393

Второе начальное приближение находится ближе к точному решению, чем первое, поэтому при нём алгоритм сходится к истинному решению за меньшее число итераций

## 4.2

In [10]:
ans = zeidel(A, b, x0_1, n_max=None, eps=10**-6)
ans[2]

3.103711732066472e-09

In [11]:
print("Решение: {}\nКоличество итераций: {}".format(ans[0], ans[1]))

Решение: [ 0.77777755  2.00000053 -2.99999948 -1.21212151]
Количество итераций: 45


## 4.3

In [12]:
ans = zeidel(A, b, x0_1, n_max=10, eps=10**-12)  # ||x_{n} - x_{n-1}||
delta = ans[2]
B, c = modify(A, b)
B2 = get_b2(B)
err_theor = norm(B2, ord=np.inf)/(1-norm(B, ord=np.inf))*delta
err_real = norm(ans[0] - x_prec, ord = np.inf)
print("Теоритическая погрешность: {}\nРеальная погрешность: {}".format(err_theor, err_real))

Теоритическая погрешность: 5.8392865283368565
Реальная погрешность: 0.03313858610631426


Известно, что при выполнении условия $\|B\|_\infty < 1$ метод Зейделя сходится со скоростью геометрической прогрессии при любом начальном приближении. 
При этом апостериорная оценка погрешности на шаге n:

$\|x_n - \overline{x}\|_\infty \le \frac{\|B\|_\infty}{1 - \|B_2\|_\infty} \|x_n - x_{n-1}\|_\infty$


## 4.6.2

Следующие два класса реализуют матрицу A и вектор b из условия

In [13]:
class SparseDiagMatrix():
    def __init__(self):
        self.size = 35
    def __getitem__(self, i):
        if type(i) == int:
            if i >= self.size:
                raise IndexError()
            row_i = [0 for i in range(self.size)]
            row_i[i] = 220
            if i < self.size - 1:
                row_i[i+1] = 22
            if i < self.size - 4:
                row_i[i+4] = 2
            return row_i
        else:
            j = i[1]
            i = i[0]   
            
            if i >= self.size or j >= self.size:
                raise IndexError()
            if i == j:
                return 220
            elif j - i == 1:
                return 22
            elif j - i == 4:
                return 2
            else:
                return 0
    def __len__(self):
        return self.size 
class b_i():
    def __init__(self):
        self.size = 35
    def __getitem__(self, i):
        if type(i) == int:
            if i >= self.size:
                raise IndexError()
            return (i+1)*np.exp(22/(i+1))*np.cos(11/(i+1))               
    def __len__(self):
        return self.size

Данный класс реализует метод CRS для хранения разреженных матриц 

In [14]:
class MatrixCRS():   #compressed row storage
    def __init__(self, matrix, zeros=False):      
        self.m = len(matrix)
        self.n = len(matrix[0])
        if not zeros:
            self.values, self.col, self.rowIndex = MatrixCRS.get_crs(matrix)
        else:
            self.values, self.col, self.rowIndex = [], [], [0 for _ in range(self.m+1)]
    def __getitem__(self, i):
        if type(i) == int:
            if i >= self.m:
                raise IndexError()
            row_i = [0 for i in range(self.n)]
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            for i in range(val_ind_begin, val_ind_end):
                row_i[self.col[i]] = self.values[i]
            return row_i
        else:
            j = i[1]
            i = i[0]
            if i >= self.m or j >= self.n:
                raise IndexError()
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            for i in range(val_ind_begin, val_ind_end):
                if self.col[i] == j:
                    return self.values[i]
            return 0  
    def __setitem__(self, i, value):
        if type(i) == int:
            if i >= self.m:
                raise IndexError()
        else:                
            j = i[1]
            i = i[0]
            if i >= self.m or j >= self.n:
                raise IndexError()
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            if val_ind_begin == val_ind_end:  ##елси нулевая строка
                if value == 0:
                    return
                k = val_ind_begin-1
                self.values = self.values[:k+1] + [value] + self.values[k+1:]
                self.col = self.col[:k+1] + [j] + self.col[k+1:]
                for l in range(i+1, self.m+1):
                    self.rowIndex[l] += 1
                return
            for k in range(val_ind_begin, val_ind_end):                
                if self.col[k] == j:        ##если такой элемент уже был в матрице ненулевым
                    if value == 0:
                        self.values = self.values[:k] + self.values[k+1:]
                        self.col = self.col[:k] + self.col[k+1:]
                        for l in range(i+1, self.m+1):
                            self.rowIndex[l] -= 1
                    else:
                        self.values[k] = value
                    return
                else:
                    if (k == val_ind_end - 1 or (k!=(val_ind_end-1) and self.col[k] < j and self.col[k+1] > j)):
                        if value == 0:
                            return
                        self.values = self.values[:k+1] + [value] + self.values[k+1:]
                        self.col = self.col[:k+1] + [j] + self.col[k+1:]
                        for l in range(i+1, self.m+1):
                            self.rowIndex[l] += 1
                        return      
    def __len__(self):
        return self.m
    def get_crs(mat): 
        m = len(mat)
        n = len(mat[0])
        val = []
        col = []
        rowId = []        
        for i in range(m):
            rowId+=[len(val)]
            for j in range(n):
                if mat[i][j] != 0:
                    val += [copy.deepcopy(mat[i][j])]
                    col += [j]
        rowId += [len(val)]
        return val, col, rowId

Модификация метода Зейделя для работы с разреженными матрицами

In [15]:
def modify(A, b, sparse=False):
    n = len(A)
    if sparse:
        B = MatrixCRS(A, zeros=True)
    else:
        B = np.zeros(shape=(n,n))
    c = np.zeros(shape=(n,))
    for i in range(n):
        c[i] = b[i]/A[i, i]
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j]/A[i, i]
    return B, c
def get_b2(B, sparse=False):
    if sparse:
        B2 = MatrixCRS(B)
    else:
        B2 = copy.deepcopy(B)
    m = len(B2)
    for i in range(m):
        for j in range(i+1):
            B2[i, j] = 0
    return B2
def zeidel(A, b, x0, n_max, eps, sparse=False):
    B, c = modify(A, b, sparse=sparse)
    m = x0.shape[0]   
    x = np.zeros(m)
    x_prev = copy.deepcopy(x0)    
    n_iter = 0
    n_iter_prev = -1
    B2 = get_b2(B, sparse=sparse)
    eps = (1-norm(B, ord=np.inf))/norm(B2, ord=np.inf)*eps
    while(n_iter != n_max):
        n_iter_prev = n_iter
        n_iter+=1
        assert (n_iter<sys.maxsize-1), "Too many iterations without convergence"
        for i in range(m):
            x[i] = sum([B[i, j]*x[j] for j in range(0, i)]) + \
            sum([B[i, j]*x_prev[j] for j in range(i+1, m)]) + \
            c[i]
        delta = norm(x - x_prev, ord=np.inf)
        if delta < eps:
            return x, n_iter, delta
        x_prev = copy.deepcopy(x)    
    return x, n_iter, delta

In [16]:
A = SparseDiagMatrix()
b = b_i()
x_prec = solve(A, b)

In [17]:
x0 = np.array([1 for i in range(35)]).reshape([-1,])
ans = zeidel(A, b, x0, n_max=None, eps=10**-9, sparse=True)
print("Приближенное решение:\n {}".format(x_prec))
delta = ans[2]
B, c = modify(A, b, sparse=True)
B2 = get_b2(B, sparse=True)
err_theor = norm(B2, ord=np.inf)/(1-norm(B, ord=np.inf))*delta
err_real = norm(ans[0] - x_prec, ord = np.inf)
print("Теоритическая погрешность: {}\nРеальная погрешность: {}".format(err_theor, err_real))
print("Количество итераций: {}".format(ans[1]))

Приближенное решение:
 [ 7.20782663e+04  3.87504620e+02 -1.76578547e+01 -4.00673285e+00
 -1.06297804e+00 -2.77261700e-01 -1.15650096e-02  9.47020608e-02
  1.42645524e-01  1.66390314e-01  1.79156571e-01  1.86625267e-01
  1.91437913e-01  1.94898786e-01  1.97681563e-01  2.00145548e-01
  2.02486191e-01  2.04810463e-01  2.07176243e-01  2.09613627e-01
  2.12136808e-01  2.14750877e-01  2.17455568e-01  2.20248228e-01
  2.23124126e-01  2.26076351e-01  2.29109478e-01  2.32180458e-01
  2.35325233e-01  2.38615671e-01  2.41361903e-01  2.47254829e-01
  2.50929944e-01  2.51487778e-01  2.83673616e-01]
Теоритическая погрешность: 2.423339352315786e-10
Реальная погрешность: 1.922444425872527e-10
Количество итераций: 10
